# Advent of Code 2024 Solutions

First, let's import all the packages we'll use:

In [1]:
import numpy as np
import re
import networkx as nx
import itertools
import math
import copy

# [Day 1 Historian Hysteria](https://adventofcode.com/2024/day/1)

In [7]:
file = open('input1.txt', 'r')
input_text = file.read()
file.close()

## Part 1

Starting off pretty easy - let's just convert our lists to vectors, sort by size, and then take the difference.

In [8]:
def create_lists(input_text):
    lines = input_text.strip().splitlines()
    left_list = []
    right_list = []
    for line in lines:
        left, right = line.split()
        left_list.append(int(left))
        right_list.append(int(right))
    return left_list, right_list

In [11]:
left_list, right_list = create_lists(input_text)
left_list = np.sort(left_list)
right_list = np.sort(right_list)
np.sum(np.absolute(right_list-left_list))

2367773

## Part 2

This one's also pretty chill. We can just loop through each value in left list and count how many times it appears in the right list.

In [13]:
score = 0
for num in left_list:
    score += num * np.count_nonzero(right_list == num)
score

21271939

# [Day 2: Red-Nosed Reports](https://adventofcode.com/2024/day/2)

In [1]:
file = open('input2.txt', 'r')
input_text = file.read()
file.close()

## Part 1

In [2]:
def read_reports(input_text):
    lines = input_text.strip().splitlines()
    
    reports = []
    for line in lines:
        reports.append([int(level) for level in line.split()])
    return reports

In [3]:
reports = read_reports(input_text)

A report is safe if its **strictly monotonic**, and each consecutive difference is **between $1$ and $3$ in absolute value**. We'll create a function to test for this and then run this function on every report in our list.

In [4]:
def is_report_safe(report):
    differences = []
    for i in range(len(report) - 1):
        differences.append(report[i+1]-report[i])
    strictly_monotonic = all(num < 0 for num in differences) or all(num > 0 for num in differences)
    gradual = all(1 <= num and 3 >= num for num in [abs(difference) for difference in differences])
    return strictly_monotonic and gradual

In [5]:
number_safe = 0
for report in reports:
    if is_report_safe(report):
        number_safe += 1
number_safe

502

## Part 2 

For each report, we can see if it only has one bad level by removing each level and seeing if the remaining levels constitute a safe report.

In [7]:
number_safe = 0
for report in reports:
    for i in range(len(report)):
        if is_report_safe([level for j, level in enumerate(report) if j != i]):
            number_safe += 1
            break
number_safe

544

# [Day 3: Mull It Over](https://adventofcode.com/2024/day/3)

In [2]:
file = open('input3.txt', 'r')
input_text = file.read()
file.close()

## Part 1

We can use a simple **regular expression** to detect all substrings of the form `mul(X,Y)` where `X` and `Y` are 1-3 digit numbers

In [3]:
mem = input_text

In [4]:
regex = r"mul\((-?\d+),(-?\d+)\)"
def extract_multiplications(mem):
    total = 0
    matches = re.findall(regex, mem)
    for match in matches:
        total += int(match[0])*int(match[1])
    return total

In [6]:
# mult_regex = r"mul\((-?\d+),(-?\d+)\)"
extract_multiplications(mem)

167090022

## Part 2

We just have to preprocess our string a bit for part 2. We can split our string into blocks separated by `don't()`'s. Since the multiplication instructions are **enabled at the beginning**, we can count all the multiplications in the first block. In the other blocks, we only want to count multiplications **after the first `do()`**, since the start of each block follows a `don't()`.

In [7]:
blocks = mem.split("don't()")
total = 0
for block in blocks[1:]:
    do = str(block.split("do()", 1)[1:])
    total += extract_multiplications(do)
total += extract_multiplications(blocks[0])
total

89823704

# [Day 4: Ceres Search](https://adventofcode.com/2024/day/4)

In [2]:
file = open('input4.txt', 'r')
input_text = file.read()
file.close()

## Part 1

In [10]:
def read_word_search(input_text):
    lines = input_text.strip().splitlines()

    # We'll pad our array so when we use numpy vector as indices, we don't have to worry about going out of bounds.
    word_search = [['.']*(len(lines[0])+2)] 
    
    for line in lines:
        row = ['.']
        for col_index, char in enumerate(line):
            row.append(char) 
        row.append('.')
        word_search.append(row)
    word_search.append(['.']*(len(lines[0])+2))
    return word_search

In [4]:
word_search = read_word_search(input_text)

For each character in our word search, we'll iterate in each direction and check if those characters spell "XMAS."

In [5]:
def detect_xmas(i, j, direction, word_search):
    current = np.array([i,j])
    for i in range(4):
        ith_letter = current + i * direction
        if word_search[ith_letter[0]][ith_letter[1]] != "XMAS"[i]:
            return 0
    return 1

In [6]:
directions = [np.array(direction) for direction in [(-1,0),(0,1),(1,0),(0,-1),(1,1),(1,-1),(-1,-1),(-1,1)]]    
total = 0
for i, row in enumerate(word_search):
    for j, entry in enumerate(row):
        if entry == '.':
            continue
        for direction in directions:
            total += detect_xmas(i, j, direction, word_search)
total

2406

## Part 2

Not much extra stuff to do here. We just have to get a little fancier to get the neighboring letters. For any given `A`, we just have to check the cells neighboring it diagonally. Below, we flatten out our diagonals from the bottom left to the top right. So if we have a diagonal crossing that looks like,

$\begin{bmatrix} x_1 & . & x_2 \\ . & x_3 & . \\ x_4 & . & x_5 \end{bmatrix}$, 

we map this to the string $x_4x_5x_3x_1x_2$. We then check to see if this matches one of the four possible`X-MAS`'s.

In [7]:
valid_x_mas = {"MSAMS","MMASS","SMASM","SSAMM"}
def detect_x_mas(i, j, word_search):
    cross = ""
    parity_checker = (i+j) % 2
    for k in range(i-1, i+2):
        for l in range(j-1, j+2):
            if (k + l) % 2 == parity_checker: 
                cross += word_search[k][l]
    return 1 if cross in valid_x_mas else 0

In [8]:
x_mas_total = 0
for i, row in enumerate(word_search):
    for j, entry in enumerate(row):
        if entry == '.':
            continue
        x_mas_total += detect_x_mas(i, j, word_search)
x_mas_total

1807

# [Day 5: Print Queue](https://adventofcode.com/2024/day/5)

In [2]:
file = open('input5.txt', 'r')
input_text = file.read()
file.close()

## Part 1

In [3]:
def get_rules_and_updates(input_text):
    rules, updates = input_text.strip().split("\n\n")
    rules = rules.split('\n')
    updates = updates.split('\n')
    return rules, updates

In [5]:
rules, updates = get_rules_and_updates(input_text)
rules = set(tuple(rule.split('|')) for rule in rules)

Since the rules give us a way to order updates of pages, we can just use **quick sort** with `|` in place of `<`. Once we get a sorted list of pages according to the rules, we can check if the update matches the sorted list of pages.

*Digression: Initially I assumed that the rules would form a total ordering of the pages since this was the case in the small example that the problem gave. So my first approach was just to attempt to sort all of the pages at the beginning, which is why I thought to use quick sort. After spending some time being very confused as to why my code wasn't working, I **finally** checked the input for the first time. It was a good lesson to check my input before writing the solution for later days.*

In [6]:
def quicksort(lst):
    if len(lst) == 0:
        return []
    pivot = lst[0]
    left = []
    right = []
    for elt in lst[1:]:
        if (elt, pivot) in rules:
            left.append(elt)
        else:
            right.append(elt)
    return quicksort(left) + [pivot] + quicksort(right)

In [7]:
total = 0 
for update in updates:
    update_pages = update.split(',')
    sorted_pages = quicksort(update_pages)
    mapped_update = [sorted_pages.index(page) for page in update_pages]
    monotonic = all([mapped_update[i] < mapped_update[i+1] for i in range(len(mapped_update) - 1)])
    if monotonic:
        total += int(update_pages[int((len(mapped_update)-1)/2)])
total

5588

## Part 2

This should be pretty easy since we already sorted updates into the correct order in Part 1.

In [8]:
total = 0 
for update in updates:
    update_pages = update.split(',')
    sorted_pages = quicksort(update_pages)
    mapped_update = [sorted_pages.index(page) for page in update_pages]
    monotonic = all([mapped_update[i] < mapped_update[i+1] for i in range(len(mapped_update) - 1)])
    if not monotonic:
        total += int(sorted_pages[int((len(mapped_update)-1)/2)])
total

5331

# [Day 6: Guard Gallivant](https://adventofcode.com/2024/day/6)

In [1]:
file = open('input6.txt', 'r')
input_text = file.read()
file.close()

## Part 1

In [3]:
def read_map(input_text):
    lines = input_text.strip().splitlines()
    matrix = [['@' for i in range(len(lines[0])+2)]]

    for row_index, line in enumerate(lines):
        row = ['@']
        for col_index, char in enumerate(line):
            if char == '#':
                row.append("#")
            else:
                row.append('.')
            if char == '^':
                start_position = (row_index+1, col_index+1)
        row.append('@')
        matrix.append(row)
    matrix.append(['@' for i in range(len(lines[0])+2)])
    
    return matrix, np.array(start_position)

In [4]:
guard_map, start_position = read_map(input_text)

In [5]:
directions = [np.array(direction) for direction in [(-1,0),(0,1),(1,0),(0,-1)]]
i = 0
position = np.copy(start_position)
squares = set()
while True:
    squares.add(tuple(position))
    next_position = position + directions[i]
    if guard_map[next_position[0]][next_position[1]] == "@":
        break
    elif guard_map[next_position[0]][next_position[1]] == "#":
        i = (i+1) % 4
    else:
        position = next_position

In [6]:
len(squares)

4656

## Part 2

# [Day 7: Bridge Repair](https://adventofcode.com/2024/day/7)

In [1]:
file = open('input7.txt', 'r')
input_text = file.read()
file.close()

## Part 1

In [7]:
def read_equations(input_text):
    equations = []
    lines = input_text.strip().splitlines()    
    for equation in lines:
        data = equation.split(' ')
        test_value = int(data[0][:-1])
        nums = tuple(int(value) for value in data[1:])
        equations.append({
            'test_value': test_value,
            'nums': nums,
        })
    return equations

In [8]:
equations = read_equations(input_text)

We'll use recursion here. For a given sequence of numbers $x_1 \ x_2 \ \ldots \ x_{n-1} \ x_n$ in an equation, we compute all the possible results we could get from different combinations of operations for the sequence $x_1 \ x_2 \ \ldots \ x_{n-1}$. Then for each number in this set, we add or multiply $x_n$ to get all the possible values we could get by inserting different combinations of operations into our entire sequence. I'm not sure whether memoizing is necessary here, but why not?

In [9]:
memoization_table = {}
def get_combinations(nums):
    if len(nums) == 1:
        return set(nums)
    last = nums[-1]
    subtuple = nums[:-1]
    if subtuple in memoization_table.keys():
        rest = memoization_table[subtuple]
    else:
        rest = get_combinations(subtuple)
        memoization_table[subtuple] = rest
    return set(last + combination for combination in rest) | set(last * combination for combination in rest)

In [10]:
total = 0
for equation in equations:
    test_value = equation['test_value']
    nums = equation['nums']
    if test_value in get_combinations(nums):
        total += test_value
total

303766880536

## Part 2

We'll apply the same recursive method as above but now we throw in the concatenation operator in the inductive step.

In [11]:
memoization_table2 = {}
def get_combinations2(nums):
    if len(nums) == 1:
        return set(nums)
    last = nums[-1]
    subtuple = nums[:-1]
    if subtuple in memoization_table2.keys():
        rest = memoization_table2[subtuple]
    else:
        rest = get_combinations2(subtuple)
        memoization_table[subtuple] = rest
    return set(last + combination for combination in rest) | set(last * combination for combination in rest) | set(int(str(combination) + str(last)) for combination in rest)

In [12]:
total = 0
for equation in equations:
    test_value = equation['test_value']
    nums = equation['nums']
    if test_value in get_combinations2(nums):
        total += test_value
total

337041851384440

# [Day 8: Resonant Collinearity](https://adventofcode.com/2024/day/8)

In [2]:
file = open('input8.txt', 'r')
input_text = file.read()
file.close()

## Part 1

In [3]:
def read_grid(input_text):
    lines = input_text.strip().splitlines()
    grid = []
    for line in lines:
        row = []
        for char in line:
            row.append(char)
        grid.append(row)
    return grid

In [4]:
grid = read_grid(input_text)

First, for each unique character/frequency, we'll get a list of coordinates where that character appears.

In [6]:
def get_character_positions(grid):
    character_positions = {}
    for i, row in enumerate(grid):
        for j, value in enumerate(row):
            if value == ".":
                continue
            if value in character_positions.keys():
                character_positions[value].add((i,j))
            else:
                character_positions[value] = {(i,j)}
    return character_positions

In [7]:
character_positions = get_character_positions(grid)

We're given that an antinode only occurs when it's aligned with two antennas of the same frequency, and the distance from one of the antennas is twice as far as the distance from the other antenna. So for every pair of antennas $a_1, a_2 \in \mathbb{Z}^2$ with the same frequency, the two possible anitnodes will be $(a_2 - a_1) + a_2$ and $a_1 - (a_2 - a_1)$. Thus, for any such pair, we just check if these two coordinates are in the bounds of our grid.

In [8]:
def is_valid_coord(coordinates):
    return coordinates[0] >= 0 and coordinates[1] >= 0 and coordinates[0] < len(grid) and coordinates[1] < len(grid[0])

def get_antinodes(antennas):
    antinodes = set()
    antennas_np = [np.array(antenna) for antenna in antennas]
    pairs = itertools.combinations(antennas_np, 2)
    for combo in pairs:
        p1, p2 = combo
        difference = p2 - p1
        antinode1 = p1 - difference
        antinode2 = p2 + difference
        
        if is_valid_coord(antinode1):
            antinodes.add(tuple(antinode1))
        if is_valid_coord(antinode2):
            antinodes.add(tuple(antinode2))
    return antinodes

In [9]:
antinodes = set()
for char in character_positions.keys():
    antinodes = antinodes | get_antinodes(character_positions[char])
len(antinodes)

289

## Part 2

For part 2, antinodes can occur at **any point that is collinear** with two antennas of the same frequency. Thus, for any pair of same-frequency antennas $a_1, a_2$, we can parametrize the line they lie on to enumerate all integer points on this line. We again look at the difference vector $a_2 - a_1$, but to ensure we attain all possible integer values on this line, we need to scale down by the $\gcd$ of its $x$ and $y$ coordinates. As before, we only have to do this for points that fit within the bounds of our grid.

In [13]:
def get_antinodes_2(antennas):
    antinodes = set()
    antennas_np = [np.array(antenna) for antenna in antennas]
    pairs = itertools.combinations(antennas_np, 2)
    for combo in pairs:
        p1, p2 = combo
        difference = p2 - p1
        gcd = math.gcd(int(difference[0]), int(difference[1]))
        difference = ((1/gcd)*difference).astype(int)
        
        current = copy.deepcopy(p1)
        while is_valid_coord(current):
            antinodes.add(tuple(current))
            current += difference

        current = copy.deepcopy(p1)
        while is_valid_coord(current):
            antinodes.add(tuple(current))
            current -= difference

    return antinodes

In [14]:
antinodes = set()
for char in character_positions.keys():
    antinodes = antinodes | get_antinodes_2(character_positions[char])
len(antinodes)

1030

# Day 9

# Day 10

# Day 11

## Part 1

In [2]:
file = open('input11.txt', 'r')
input_text = file.read()
file.close()

In [3]:
stones = [int(stone) for stone in input_text.strip().split(' ')]

Since each stone changes **simultaneously** by the **same rules**, we can apply the appropriate to change to each stone individually in order and place all the results in a list. Then we just repeat this 25 times to get our final answer.

In [4]:
def change(stone):
    if stone == 0:
        return [1]
    s = str(stone)
    if len(s) % 2 == 0:
        return [int(s[:int(len(s)/2)]), int(s[int(len(s)/2):])]
    return [stone*2024]

def blink(stones):
    new_stones = []
    for stone in stones:
        new_stones += change(stone)
    return new_stones

In [5]:
stones_temp = [stone for stone in stones]
for i in range(0,25):
    stones_temp = blink(stones_temp)

In [6]:
len(stones_temp)

186203

## Part 2

Since blinking will cause the number of stones to **potentially double**, the number of times we have to run `change` grows **exponentially** in the worst case. So it's probably not the best idea to try and apply the same strategy as before 75 times. Instead, we notice that whenever we change our stones, we have the potential to produce a stone with a number we have already seen before. Thus, we can use recursion + memoization to reduce the number of times we need to call `change`. 

If we have `stones = [s1, s2, ..., sk]`, and we want to count the number of stones after $n$ blinks, we can use the recursive computation:

`count_stones(stones, n) = count_stones(s1, n-1) + count_stones(s2, n-1) + ... + counts_stones(sk, n-1)` for $n > 1$, and

`count_stones(stones, 1) = len(blink(stones))` for $n = 1$.

In [7]:
memoization_table = {}

def count_stones(stones, n):
    # If we're only blinking once, we just use our original blink method
    if n == 1:
        return len(blink(stones))
        
    total = 0
    for stone in stones:
        if (stone, n-1) in memoization_table.keys():
            total += memoization_table[(stone,n-1)]
        else:
            memoization_table[(stone,n-1)] = count_stones(change(stone), n-1)
            total += memoization_table[(stone,n-1)]
    return total

In [8]:
count_stones(stones, 75)

221291560078593

Now that we successfully implemented our recursive approach, we can see how many calls to `change` we saved.

In [9]:
len(memoization_table)

121273

Since our memoization table only has `121273` distinct values, and we memoized every value we computed, this is roughly equal to the number of times we called `change` (not accounting for the times in the `blink` calls). On the other hand, if we just recursively applied `blink`, we would have had to call `change` for every stone in our list of stones after blinks 1 through 74. So to count the number of calls to `change` we would have made in our iterative approach, we just sum up the number of stones after $i$ blinks for $1 \leq i \leq 74$.

In [10]:
total_change_calls = 0
for i in range(1, 74):
    total_change_calls += count_stones(stones, i)
total_change_calls

280740494459387

Wow, I'm a math PhD, so I can say with confidence that $280740494459387 - 121273 = 280740494282105$ is a large number of extra calls.